# ML Engineer Workflow Demo

This notebook demonstrates the typical workflow for an ML Engineer working with the MLOps SageMaker Demo project. It focuses on pipeline development, model deployment, and monitoring tasks that are accessible with the ML Engineer IAM role.

## Workflow Overview

1. **Pipeline Development**: Create and manage SageMaker Pipelines
2. **Model Deployment**: Deploy models to SageMaker endpoints
3. **Monitoring Setup**: Configure model monitoring and drift detection

## Prerequisites

- AWS account with appropriate permissions
- AWS CLI configured with "ab" profile
- SageMaker Studio access with ML Engineer role
- Access to the model registry and deployment resources

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, HTML
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Set up AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
sagemaker_session = sagemaker.Session(boto_session=session)
region = session.region_name
account_id = session.client('sts').get_caller_identity()['Account']

# Import project modules
import sys
sys.path.append('..')
from src.pipeline.sagemaker_pipeline import create_training_pipeline
from src.pipeline.model_monitor import setup_model_monitoring
from src.pipeline.mlflow_integration import MLFlowSageMakerIntegration
from src.monitoring.drift_detection import create_model_drift_alert
from src.monitoring.cost_tracking import create_model_monitoring_dashboard

# Set up visualization
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Load project configuration
from configs.project_config import PROJECT_NAME, DATA_BUCKET, ROLE_ARN

print(f"Project: {PROJECT_NAME}")
print(f"Data Bucket: {DATA_BUCKET}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")

## 1. Pipeline Development

Let's start by creating a SageMaker Pipeline for training and evaluating YOLOv11 models.

In [ ]:
# Create interactive widgets for pipeline configuration
pipeline_name = widgets.Text(
    value=f"yolov11-training-pipeline-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    description='Pipeline Name:',
    style={'description_width': 'initial'}
)

training_data_path = widgets.Text(
    value="",
    description='Training Data:',
    placeholder='s3://bucket/training-data/yolo-format/job-name/',
    style={'description_width': 'initial'}
)

validation_data_path = widgets.Text(
    value="",
    description='Validation Data:',
    placeholder='s3://bucket/validation-data/yolo-format/job-name/',
    style={'description_width': 'initial'}
)

model_variant = widgets.Dropdown(
    options=['yolov11n', 'yolov11s', 'yolov11m', 'yolov11l', 'yolov11x'],
    value='yolov11n',
    description='Model Variant:',
    style={'description_width': 'initial'}
)

instance_type = widgets.Dropdown(
    options=['ml.g4dn.xlarge', 'ml.g4dn.2xlarge', 'ml.g5.xlarge'],
    value='ml.g4dn.xlarge',
    description='Instance Type:',
    style={'description_width': 'initial'}
)

use_spot = widgets.Checkbox(
    value=True,
    description='Use Spot Instances',
    style={'description_width': 'initial'}
)

auto_deploy = widgets.Checkbox(
    value=False,
    description='Auto Deploy Model',
    style={'description_width': 'initial'}
)

map_threshold = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.05,
    description='mAP Threshold:',
    style={'description_width': 'initial'}
)

display(pipeline_name, training_data_path, validation_data_path, model_variant, instance_type, use_spot, auto_deploy, map_threshold)

In [ ]:
# Create function to create pipeline
def create_pipeline():
    # Get widget values
    name = pipeline_name.value
    train_data = training_data_path.value
    val_data = validation_data_path.value
    variant = model_variant.value
    instance = instance_type.value
    spot = use_spot.value
    deploy = auto_deploy.value
    threshold = map_threshold.value
    
    # Validate inputs
    if not train_data or not val_data:
        print("Please enter valid training and validation data paths")
        return
    
    # Create pipeline configuration
    pipeline_config = {
        "pipeline_name": name,
        "training_data": train_data,
        "validation_data": val_data,
        "model_variant": variant,
        "instance_type": instance,
        "use_spot_instances": spot,
        "auto_deploy": deploy,
        "map_threshold": threshold,
        "role_arn": ROLE_ARN,
        "output_bucket": DATA_BUCKET
    }
    
    # Create the pipeline
    try:
        pipeline = create_training_pipeline(**pipeline_config)
        
        # Create the pipeline in SageMaker
        pipeline.upsert(role_arn=ROLE_ARN)
        
        print(f"Created pipeline: {name}")
        print(f"Pipeline ARN: {pipeline.arn}")
        
        # Return the pipeline
        return pipeline
    except Exception as e:
        print(f"Error creating pipeline: {e}")
        return None

# Create button to create pipeline
create_pipeline_button = widgets.Button(
    description='Create Pipeline',
    button_style='primary',
    tooltip='Click to create the pipeline'
)

pipeline_output = widgets.Output()

def on_create_pipeline_clicked(b):
    with pipeline_output:
        pipeline_output.clear_output()
        pipeline = create_pipeline()

create_pipeline_button.on_click(on_create_pipeline_clicked)

display(create_pipeline_button, pipeline_output)

### 1.1 Execute Pipeline

Let's create a tool to execute the pipeline.

In [ ]:
# Create interactive widgets for pipeline execution
execution_pipeline_name = widgets.Text(
    value="",
    description='Pipeline Name:',
    placeholder='Enter the pipeline name',
    style={'description_width': 'initial'}
)

# Create button to execute pipeline
execute_pipeline_button = widgets.Button(
    description='Execute Pipeline',
    button_style='success',
    tooltip='Click to execute the pipeline'
)

execution_output = widgets.Output()

# Function to execute pipeline
def execute_pipeline():
    # Get pipeline name
    name = execution_pipeline_name.value
    
    # Validate input
    if not name:
        print("Please enter a valid pipeline name")
        return
    
    # Execute the pipeline
    try:
        # Create SageMaker Pipeline client
        pipeline_client = session.client('sagemaker-pipeline')
        
        # Start pipeline execution
        response = pipeline_client.start_pipeline_execution(
            PipelineName=name
        )
        
        execution_arn = response['PipelineExecutionArn']
        
        print(f"Started pipeline execution: {name}")
        print(f"Execution ARN: {execution_arn}")
        
        # Return the execution ARN
        return execution_arn
    except Exception as e:
        print(f"Error executing pipeline: {e}")
        return None

def on_execute_pipeline_clicked(b):
    with execution_output:
        execution_output.clear_output()
        execution_arn = execute_pipeline()

execute_pipeline_button.on_click(on_execute_pipeline_clicked)

display(execution_pipeline_name, execute_pipeline_button, execution_output)

### 1.2 List Pipelines

Let's create a tool to list existing pipelines.

In [ ]:
# Create button to list pipelines
list_pipelines_button = widgets.Button(
    description='List Pipelines',
    button_style='info',
    tooltip='Click to list existing pipelines'
)

list_pipelines_output = widgets.Output()

# Function to list pipelines
def list_pipelines():
    try:
        # Create SageMaker Pipeline client
        pipeline_client = session.client('sagemaker-pipeline')
        
        # List pipelines
        response = pipeline_client.list_pipelines()
        
        # Display pipelines
        if 'PipelineSummaries' in response and response['PipelineSummaries']:
            pipelines = response['PipelineSummaries']
            
            # Create a DataFrame for better display
            pipeline_data = []
            for pipeline in pipelines:
                pipeline_data.append({
                    'Name': pipeline['PipelineName'],
                    'ARN': pipeline['PipelineArn'],
                    'Created': pipeline['CreationTime'].strftime('%Y-%m-%d %H:%M:%S'),
                    'Last Modified': pipeline.get('LastModifiedTime', '').strftime('%Y-%m-%d %H:%M:%S') if pipeline.get('LastModifiedTime') else ''
                })
            
            df = pd.DataFrame(pipeline_data)
            display(df)
            
            # Create a dropdown to select a pipeline
            pipeline_selector = widgets.Dropdown(
                options=[pipeline['PipelineName'] for pipeline in pipelines],
                description='Select Pipeline:',
                style={'description_width': 'initial'}
            )
            
            # Function to select pipeline
            def select_pipeline(change):
                execution_pipeline_name.value = change['new']
            
            pipeline_selector.observe(select_pipeline, names='value')
            
            display(pipeline_selector)
            
            return pipelines
        else:
            print("No pipelines found")
            return []
    except Exception as e:
        print(f"Error listing pipelines: {e}")
        return []

def on_list_pipelines_clicked(b):
    with list_pipelines_output:
        list_pipelines_output.clear_output()
        pipelines = list_pipelines()

list_pipelines_button.on_click(on_list_pipelines_clicked)

display(list_pipelines_button, list_pipelines_output)

## 2. Model Deployment

Now, let's deploy a model to a SageMaker endpoint.

In [ ]:
# Create interactive widgets for model deployment
model_package_arn = widgets.Text(
    value="",
    description='Model Package ARN:',
    placeholder='Enter the model package ARN',
    style={'description_width': 'initial'}
)

endpoint_name = widgets.Text(
    value=f"yolov11-endpoint-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    description='Endpoint Name:',
    style={'description_width': 'initial'}
)

endpoint_instance_type = widgets.Dropdown(
    options=['ml.g4dn.xlarge', 'ml.g4dn.2xlarge', 'ml.g5.xlarge', 'ml.inf1.xlarge'],
    value='ml.g4dn.xlarge',
    description='Instance Type:',
    style={'description_width': 'initial'}
)

instance_count = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Instance Count:',
    style={'description_width': 'initial'}
)

enable_autoscaling = widgets.Checkbox(
    value=True,
    description='Enable Auto-scaling',
    style={'description_width': 'initial'}
)

min_instances = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Min Instances:',
    style={'description_width': 'initial'}
)

max_instances = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Max Instances:',
    style={'description_width': 'initial'}
)

display(model_package_arn, endpoint_name, endpoint_instance_type, instance_count, enable_autoscaling, min_instances, max_instances)

In [ ]:
# Create function to deploy model
def deploy_model():
    from sagemaker.model import Model
    from sagemaker.model_package import ModelPackage
    
    # Get widget values
    model_arn = model_package_arn.value
    endpoint = endpoint_name.value
    instance_type = endpoint_instance_type.value
    count = instance_count.value
    autoscaling = enable_autoscaling.value
    min_count = min_instances.value
    max_count = max_instances.value
    
    # Validate inputs
    if not model_arn or not endpoint:
        print("Please enter valid model package ARN and endpoint name")
        return
    
    try:
        # Create model from model package
        model = ModelPackage(
            model_package_arn=model_arn,
            role=ROLE_ARN,
            sagemaker_session=sagemaker_session
        )
        
        # Deploy the model to an endpoint
        predictor = model.deploy(
            initial_instance_count=count,
            instance_type=instance_type,
            endpoint_name=endpoint
        )
        
        print(f"Deployed model to endpoint: {endpoint}")
        
        # Configure auto-scaling if enabled
        if autoscaling:
            # Create auto-scaling client
            autoscaling_client = session.client('application-autoscaling')
            
            # Register the endpoint as a scalable target
            autoscaling_client.register_scalable_target(
                ServiceNamespace='sagemaker',
                ResourceId=f'endpoint/{endpoint}/variant/AllTraffic',
                ScalableDimension='sagemaker:variant:DesiredInstanceCount',
                MinCapacity=min_count,
                MaxCapacity=max_count
            )
            
            # Configure scaling policy
            autoscaling_client.put_scaling_policy(
                ServiceNamespace='sagemaker',
                ResourceId=f'endpoint/{endpoint}/variant/AllTraffic',
                ScalableDimension='sagemaker:variant:DesiredInstanceCount',
                PolicyName=f'{endpoint}-autoscaling-policy',
                PolicyType='TargetTrackingScaling',
                TargetTrackingScalingPolicyConfiguration={
                    'TargetValue': 70.0,  # Target 70% CPU utilization
                    'PredefinedMetricSpecification': {
                        'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance',
                    },
                    'ScaleInCooldown': 300,  # 5 minutes
                    'ScaleOutCooldown': 300  # 5 minutes
                }
            )
            
            print(f"Configured auto-scaling for endpoint: {endpoint}")
            print(f"Min instances: {min_count}, Max instances: {max_count}")
        
        return predictor
    except Exception as e:
        print(f"Error deploying model: {e}")
        return None

# Create button to deploy model
deploy_button = widgets.Button(
    description='Deploy Model',
    button_style='success',
    tooltip='Click to deploy the model'
)

deploy_output = widgets.Output()

def on_deploy_clicked(b):
    with deploy_output:
        deploy_output.clear_output()
        predictor = deploy_model()

deploy_button.on_click(on_deploy_clicked)

display(deploy_button, deploy_output)

### 2.1 List Model Packages

Let's create a tool to list available model packages.

In [ ]:
# Create button to list model packages
list_models_button = widgets.Button(
    description='List Model Packages',
    button_style='info',
    tooltip='Click to list available model packages'
)

list_models_output = widgets.Output()

# Function to list model packages
def list_model_packages():
    try:
        # Create SageMaker client
        sagemaker_client = session.client('sagemaker')
        
        # List model packages
        response = sagemaker_client.list_model_packages()
        
        # Display model packages
        if 'ModelPackageSummaryList' in response and response['ModelPackageSummaryList']:
            model_packages = response['ModelPackageSummaryList']
            
            # Create a DataFrame for better display
            model_data = []
            for model in model_packages:
                model_data.append({
                    'Name': model['ModelPackageName'],
                    'ARN': model['ModelPackageArn'],
                    'Status': model['ModelPackageStatus'],
                    'Created': model['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
            
            df = pd.DataFrame(model_data)
            display(df)
            
            # Create a dropdown to select a model package
            model_selector = widgets.Dropdown(
                options=[(model['ModelPackageName'], model['ModelPackageArn']) for model in model_packages],
                description='Select Model:',
                style={'description_width': 'initial'}
            )
            
            # Function to select model package
            def select_model(change):
                model_package_arn.value = change['new']
            
            model_selector.observe(select_model, names='value')
            
            display(model_selector)
            
            return model_packages
        else:
            print("No model packages found")
            return []
    except Exception as e:
        print(f"Error listing model packages: {e}")
        return []

def on_list_models_clicked(b):
    with list_models_output:
        list_models_output.clear_output()
        model_packages = list_model_packages()

list_models_button.on_click(on_list_models_clicked)

display(list_models_button, list_models_output)

### 2.2 List Endpoints

Let's create a tool to list existing endpoints.

In [ ]:
# Create button to list endpoints
list_endpoints_button = widgets.Button(
    description='List Endpoints',
    button_style='info',
    tooltip='Click to list existing endpoints'
)

list_endpoints_output = widgets.Output()

# Function to list endpoints
def list_endpoints():
    try:
        # Create SageMaker client
        sagemaker_client = session.client('sagemaker')
        
        # List endpoints
        response = sagemaker_client.list_endpoints()
        
        # Display endpoints
        if 'Endpoints' in response and response['Endpoints']:
            endpoints = response['Endpoints']
            
            # Create a DataFrame for better display
            endpoint_data = []
            for endpoint in endpoints:
                endpoint_data.append({
                    'Name': endpoint['EndpointName'],
                    'ARN': endpoint['EndpointArn'],
                    'Status': endpoint['EndpointStatus'],
                    'Created': endpoint['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
            
            df = pd.DataFrame(endpoint_data)
            display(df)
            
            # Create a dropdown to select an endpoint
            endpoint_selector = widgets.Dropdown(
                options=[endpoint['EndpointName'] for endpoint in endpoints],
                description='Select Endpoint:',
                style={'description_width': 'initial'}
            )
            
            # Function to select endpoint
            def select_endpoint(change):
                monitoring_endpoint_name.value = change['new']
            
            endpoint_selector.observe(select_endpoint, names='value')
            
            display(endpoint_selector)
            
            return endpoints
        else:
            print("No endpoints found")
            return []
    except Exception as e:
        print(f"Error listing endpoints: {e}")
        return []

def on_list_endpoints_clicked(b):
    with list_endpoints_output:
        list_endpoints_output.clear_output()
        endpoints = list_endpoints()

list_endpoints_button.on_click(on_list_endpoints_clicked)

display(list_endpoints_button, list_endpoints_output)

## 3. Model Monitoring and Drift Detection

Finally, let's set up model monitoring for the deployed endpoint.

In [ ]:
# Create interactive widgets for monitoring configuration
monitoring_endpoint_name = widgets.Text(
    value="",
    description='Endpoint Name:',
    placeholder='Enter the deployed endpoint name',
    style={'description_width': 'initial'}
)

baseline_path = widgets.Text(
    value="",
    description='Baseline Data:',
    placeholder='s3://bucket/baseline-data/',
    style={'description_width': 'initial'}
)

monitoring_schedule = widgets.Text(
    value="",
    description='Schedule Name:',
    placeholder='Enter a name for the monitoring schedule',
    style={'description_width': 'initial'}
)

monitoring_instance = widgets.Dropdown(
    options=['ml.t3.medium', 'ml.m5.large', 'ml.m5.xlarge'],
    value='ml.t3.medium',
    description='Instance Type:',
    style={'description_width': 'initial'}
)

schedule_expression = widgets.Text(
    value="cron(0 * ? * * *)",  # Hourly schedule
    description='Schedule:',
    placeholder='cron(0 * ? * * *)',
    style={'description_width': 'initial'}
)

display(monitoring_endpoint_name, baseline_path, monitoring_schedule, monitoring_instance, schedule_expression)

In [ ]:
# Create function to set up model monitoring
def setup_monitoring():
    # Get widget values
    endpoint = monitoring_endpoint_name.value
    baseline = baseline_path.value
    schedule = monitoring_schedule.value
    instance = monitoring_instance.value
    cron_expression = schedule_expression.value
    
    # Validate inputs
    if not endpoint or not baseline or not schedule:
        print("Please enter valid endpoint name, baseline data path, and schedule name")
        return
    
    try:
        # Set up model monitoring
        setup_model_monitoring(
            endpoint_name=endpoint,
            baseline_dataset=baseline,
            monitoring_schedule_name=schedule,
            instance_type=instance,
            schedule_expression=cron_expression,
            session=session
        )
        
        print(f"Set up model monitoring for endpoint: {endpoint}")
        print(f"Monitoring schedule: {schedule}")
        print(f"Schedule expression: {cron_expression}")
        
        return schedule
    except Exception as e:
        print(f"Error setting up monitoring: {e}")
        return None

# Create button to set up monitoring
monitoring_button = widgets.Button(
    description='Setup Monitoring',
    button_style='primary',
    tooltip='Click to set up model monitoring'
)

monitoring_output = widgets.Output()

def on_monitoring_clicked(b):
    with monitoring_output:
        monitoring_output.clear_output()
        schedule = setup_monitoring()

monitoring_button.on_click(on_monitoring_clicked)

display(monitoring_button, monitoring_output)

### 3.1 Configure Drift Alerts

Let's set up alerts for model drift detection.

In [ ]:
# Create interactive widgets for alert configuration
alert_endpoint_name = widgets.Text(
    value="",
    description='Endpoint Name:',
    placeholder='Enter the endpoint name',
    style={'description_width': 'initial'}
)

metric_name = widgets.Dropdown(
    options=['data_drift_score', 'prediction_drift_score', 'accuracy_score'],
    value='data_drift_score',
    description='Metric:',
    style={'description_width': 'initial'}
)

threshold = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Threshold:',
    style={'description_width': 'initial'}
)

evaluation_period = widgets.IntSlider(
    value=3600,
    min=300,
    max=86400,
    step=300,
    description='Eval Period (s):',
    style={'description_width': 'initial'}
)

notification_email = widgets.Text(
    value="",
    description='Email:',
    placeholder='Enter notification email',
    style={'description_width': 'initial'}
)

display(alert_endpoint_name, metric_name, threshold, evaluation_period, notification_email)

In [ ]:
# Create function to set up drift alerts
def setup_drift_alerts():
    # Get widget values
    endpoint = alert_endpoint_name.value
    metric = metric_name.value
    thresh = threshold.value
    eval_period = evaluation_period.value
    email = notification_email.value
    
    # Validate inputs
    if not endpoint or not email:
        print("Please enter valid endpoint name and notification email")
        return
    
    try:
        # Create SNS topic for notifications
        sns_client = session.client('sns')
        topic_name = f"model-drift-alerts-{endpoint}"
        
        # Create topic
        topic_response = sns_client.create_topic(Name=topic_name)
        topic_arn = topic_response['TopicArn']
        
        # Subscribe email to topic
        sns_client.subscribe(
            TopicArn=topic_arn,
            Protocol='email',
            Endpoint=email
        )
        
        print(f"Created SNS topic: {topic_arn}")
        print(f"Subscribed email: {email}")
        
        # Create model drift alert
        create_model_drift_alert(
            endpoint_name=endpoint,
            metric_name=metric,
            threshold=thresh,
            evaluation_period=eval_period,
            notification_topic=topic_arn,
            session=session
        )
        
        print(f"Created drift alert for endpoint: {endpoint}")
        print(f"Metric: {metric}, Threshold: {thresh}, Evaluation Period: {eval_period} seconds")
        
        return topic_arn
    except Exception as e:
        print(f"Error setting up drift alerts: {e}")
        return None

# Create button to set up alerts
alert_button = widgets.Button(
    description='Setup Alerts',
    button_style='warning',
    tooltip='Click to set up drift alerts'
)

alert_output = widgets.Output()

def on_alert_clicked(b):
    with alert_output:
        alert_output.clear_output()
        topic_arn = setup_drift_alerts()

alert_button.on_click(on_alert_clicked)

display(alert_button, alert_output)

### 3.2 Create Monitoring Dashboard

Let's create a CloudWatch dashboard for model monitoring.

In [ ]:
# Create interactive widgets for dashboard configuration
dashboard_endpoint_name = widgets.Text(
    value="",
    description='Endpoint Name:',
    placeholder='Enter the endpoint name',
    style={'description_width': 'initial'}
)

dashboard_name = widgets.Text(
    value="",
    description='Dashboard Name:',
    placeholder='Enter dashboard name',
    style={'description_width': 'initial'}
)

display(dashboard_endpoint_name, dashboard_name)

In [ ]:
# Create function to create monitoring dashboard
def create_dashboard():
    # Get widget values
    endpoint = dashboard_endpoint_name.value
    dashboard = dashboard_name.value
    
    # Validate inputs
    if not endpoint or not dashboard:
        print("Please enter valid endpoint name and dashboard name")
        return
    
    try:
        # Create monitoring dashboard
        create_model_monitoring_dashboard(
            endpoint_name=endpoint,
            dashboard_name=dashboard,
            session=session
        )
        
        print(f"Created monitoring dashboard: {dashboard}")
        print(f"Dashboard URL: https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#dashboards:name={dashboard}")
        
        return dashboard
    except Exception as e:
        print(f"Error creating dashboard: {e}")
        return None

# Create button to create dashboard
dashboard_button = widgets.Button(
    description='Create Dashboard',
    button_style='success',
    tooltip='Click to create monitoring dashboard'
)

dashboard_output = widgets.Output()

def on_dashboard_clicked(b):
    with dashboard_output:
        dashboard_output.clear_output()
        dashboard = create_dashboard()

dashboard_button.on_click(on_dashboard_clicked)

display(dashboard_button, dashboard_output)

## 4. Automated Retraining Setup

Let's set up automated retraining based on drift detection.

In [ ]:
# Create interactive widgets for automated retraining configuration
retraining_endpoint_name = widgets.Text(
    value="",
    description='Endpoint Name:',
    placeholder='Enter the endpoint name',
    style={'description_width': 'initial'}
)

drift_threshold = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Drift Threshold:',
    style={'description_width': 'initial'}
)

pipeline_name = widgets.Text(
    value="",
    description='Pipeline Name:',
    placeholder='Enter the pipeline name',
    style={'description_width': 'initial'}
)

approval_required = widgets.Checkbox(
    value=True,
    description='Require Approval',
    style={'description_width': 'initial'}
)

approval_email = widgets.Text(
    value="",
    description='Approval Email:',
    placeholder='Enter approval notification email',
    style={'description_width': 'initial'}
)

display(retraining_endpoint_name, drift_threshold, pipeline_name, approval_required, approval_email)

In [ ]:
# Create function to set up automated retraining
def setup_automated_retraining():
    from src.training.automated_retraining_trigger import setup_automated_retraining_trigger
    
    # Get widget values
    endpoint = retraining_endpoint_name.value
    threshold = drift_threshold.value
    pipeline = pipeline_name.value
    approval = approval_required.value
    email = approval_email.value
    
    # Validate inputs
    if not endpoint or not pipeline:
        print("Please enter valid endpoint name and pipeline name")
        return
    
    if approval and not email:
        print("Please enter an approval notification email")
        return
    
    try:
        # Set up automated retraining
        trigger_arn = setup_automated_retraining_trigger(
            endpoint_name=endpoint,
            drift_threshold=threshold,
            pipeline_name=pipeline,
            require_approval=approval,
            approval_email=email if approval else None,
            session=session
        )
        
        print(f"Set up automated retraining for endpoint: {endpoint}")
        print(f"Drift threshold: {threshold}")
        print(f"Pipeline to trigger: {pipeline}")
        if approval:
            print(f"Approval required, notifications will be sent to: {email}")
        else:
            print("No approval required, retraining will start automatically")
        
        return trigger_arn
    except Exception as e:
        print(f"Error setting up automated retraining: {e}")
        return None

# Create button to set up automated retraining
retraining_button = widgets.Button(
    description='Setup Automated Retraining',
    button_style='danger',
    tooltip='Click to set up automated retraining'
)

retraining_output = widgets.Output()

def on_retraining_clicked(b):
    with retraining_output:
        retraining_output.clear_output()
        trigger_arn = setup_automated_retraining()

retraining_button.on_click(on_retraining_clicked)

display(retraining_button, retraining_output)

## 5. Resource Cleanup

Let's create a tool to clean up resources when they're no longer needed.

In [ ]:
# Create function to clean up resources
def cleanup_resources():
    import boto3
    
    sagemaker_client = session.client('sagemaker')
    cloudwatch_client = session.client('cloudwatch')
    sns_client = session.client('sns')
    
    # Get endpoint name
    endpoint = monitoring_endpoint_name.value or alert_endpoint_name.value or dashboard_endpoint_name.value or retraining_endpoint_name.value
    
    if endpoint:
        # Delete monitoring schedule
        schedule = monitoring_schedule.value
        if schedule:
            try:
                sagemaker_client.delete_monitoring_schedule(
                    MonitoringScheduleName=schedule
                )
                print(f"Deleted monitoring schedule: {schedule}")
            except Exception as e:
                print(f"Error deleting monitoring schedule: {e}")
        
        # Delete endpoint
        try:
            sagemaker_client.delete_endpoint(EndpointName=endpoint)
            print(f"Deleted endpoint: {endpoint}")
        except Exception as e:
            print(f"Error deleting endpoint: {e}")
    
    # Delete pipeline
    pipeline = pipeline_name.value
    if pipeline:
        try:
            sagemaker_client.delete_pipeline(PipelineName=pipeline)
            print(f"Deleted pipeline: {pipeline}")
        except Exception as e:
            print(f"Error deleting pipeline: {e}")
    
    # Delete dashboard
    dashboard = dashboard_name.value
    if dashboard:
        try:
            cloudwatch_client.delete_dashboards(DashboardNames=[dashboard])
            print(f"Deleted dashboard: {dashboard}")
        except Exception as e:
            print(f"Error deleting dashboard: {e}")
    
    print("Cleanup complete")

# Create button to clean up resources
cleanup_button = widgets.Button(
    description='Cleanup Resources',
    button_style='danger',
    tooltip='Click to clean up all resources'
)

cleanup_output = widgets.Output()

def on_cleanup_clicked(b):
    with cleanup_output:
        cleanup_output.clear_output()
        cleanup_resources()

cleanup_button.on_click(on_cleanup_clicked)

display(cleanup_button, cleanup_output)

## Conclusion

In this notebook, we've demonstrated the typical workflow for an ML Engineer working with the MLOps SageMaker Demo project. We've covered:

1. **Pipeline Development**: Creating and managing SageMaker Pipelines
2. **Model Deployment**: Deploying models to SageMaker endpoints with auto-scaling
3. **Monitoring Setup**: Configuring model monitoring, drift detection, and alerts
4. **Automated Retraining**: Setting up automated retraining based on drift detection
5. **Resource Cleanup**: Cleaning up resources when they're no longer needed

This workflow showcases the MLOps capabilities of Amazon SageMaker for computer vision applications.